# ELISA - Explore the Data

## Python Environment

In [1]:
import os
import pandas as pd
import seaborn as sns
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import (Column, DateTime, Integer, Numeric, Float, String, Text)

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path
from dotenv import load_dotenv

## User Environment

In [2]:
env_path = Path('.')/'.env'
load_dotenv(".env")
DB_USER = os.environ.get('DB_USER')
DB_PASSWORD = os.environ.get('DB_PASSWORD')
DB_SERVER = os.environ.get('DB_SERVER')
DB_DATABASE = os.environ.get('DB_DATABASE')
print(DB_DATABASE)

DR29Dev


## Database Connection

In [3]:
conn = f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_SERVER}:3306/{DB_DATABASE}"
db_conn = create_engine(conn, echo=False)

# Retrieve ELISA Results

In [5]:
sql = '''
SELECT l1.immunology_symbol,
       e1.analyte_reported,
       e1.unit_reported,
       e1.unit_preferred,
       e1.value_reported,
       e1.value_preferred,
       e1.study_accession
  FROM elisa_result e1
    LEFT OUTER JOIN lk_analyte l1
      ON e1.analyte_preferred = l1.analyte_accession
LIMIT 10
'''
elisa_result = pd.read_sql(sql, db_conn)
print(elisa_result.shape)
number_of_rows = len(elisa_result)
pd.set_option("display.max_rows",number_of_rows)
elisa_result.head(10)

(10, 7)


,immunology_symbol,analyte_reported,unit_reported,unit_preferred,value_reported,value_preferred,study_accession
0,None,IgG-a Amb a,U/ml,None,114.8,114.80,SDY1
1,None,IgG-a Ragweed,ng/ml,ng/ml,100,100.00,SDY1
2,None,Free IgE Concentration,ng/ml,None,None,NaN,SDY1
3,None,IgE-a Amb a,U/ml,None,5,5.00,SDY1
4,None,IgE-a Ragweed,kIUa/ml,None,2.49,2.49,SDY1
5,None,IgG-a Amb a,U/ml,None,2112,2112.00,SDY1
6,None,IgG-a Ragweed,ng/ml,ng/ml,2606,2606.00,SDY1
7,None,Free IgE Concentration,ng/ml,None,None,NaN,SDY1
8,None,IgE-a Amb a,U/ml,None,5,5.00,SDY1
9,None,IgE-a Ragweed,kIUa/ml,None,2.22,2.22,SDY1
